In [1]:
%cd /home/parthgandhi/TradeBot

/home/parthgandhi/TradeBot


In [2]:
import polars as pl
import polars.selectors as cs
from src.config.storage_layout import StorageLayout
from src.config.market import Market
from src.config.exchange import Exchange

In [3]:
END_DATE = "2025-12-30"
RS_RATING = 70

# Industry Analysis

In [4]:
analysis_path = StorageLayout.analysis_dir(
    run_date=END_DATE, market=Market.INDIA_EQUITIES, exchange=Exchange.NSE
)

res = pl.scan_csv(analysis_path / "overall_filter_result.csv").filter(
    pl.col("rs_rating") >= RS_RATING
)

In [5]:
industry_analysis = (
    res.lazy()
    .group_by("sector_cmaze", "basic_industry_cmaze")
    .agg(
        [pl.col("symbol").count().alias("symbols_count")]
        + [
            (
                (pl.col(f"{i}_mo_rtr_pct") * pl.col("market_cap_cr")).sum()
                / pl.col("market_cap_cr_cmaze").sum()
            )
            .round(2)
            .alias(f"{i}_mo_avg_weight_rtr_pct")
            for i in [1, 3]
        ]
        + [
            (
                (pl.col("rs_rating") * pl.col("market_cap_cr_cmaze")).sum()
                / pl.col("market_cap_cr_cmaze").sum()
            )
            .round(2)
            .alias("rs_rating_avg_weight")
        ]
    )
    .fill_nan(None)
    .with_columns(
        pl.mean_horizontal(
            cs.exclude("sector_cmaze", "basic_industry_cmaze", "symbols_count")
        )
        .round(2)
        .alias("industry_score"),
        (pl.col("1_mo_avg_weight_rtr_pct") * 100 / pl.col("3_mo_avg_weight_rtr_pct"))
        .round()
        .alias("1_by_3_rtr"),
    )
    .sort("symbols_count", descending=True)
    .collect()
)

industry_analysis.write_csv(analysis_path / "industry_analysis.csv")

# RS >= 70 & Possible Upcoming Centers

In [6]:
industry_analysis.filter(
    (pl.col("rs_rating_avg_weight") >= 70)
    & (pl.col("1_mo_avg_weight_rtr_pct") >= pl.col("3_mo_avg_weight_rtr_pct"))
).sort("industry_score", descending=True)

sector_cmaze,basic_industry_cmaze,symbols_count,1_mo_avg_weight_rtr_pct,3_mo_avg_weight_rtr_pct,rs_rating_avg_weight,industry_score,1_by_3_rtr
str,str,u32,f64,f64,f64,f64,f64
"""Media Entertainment & Publicat…","""Media & Entertainment""",1,38.42,37.72,98.0,58.05,102.0
"""Capital Goods""","""Packaging""",1,22.92,6.26,95.0,41.39,366.0
"""Chemicals""","""Petrochemicals""",1,34.49,12.92,73.0,40.14,267.0
"""Capital Goods""","""Electrical - Power Equipment""",5,11.78,8.63,93.9,38.1,137.0
"""Oil Gas & Consumable Fuels""","""Oil & Gas Drilling""",1,7.14,4.23,84.0,31.79,169.0
"""Services""","""Logistics""",2,12.15,7.09,74.58,31.27,171.0
"""FMCG""","""Tea & Coffee""",2,0.42,0.12,89.73,30.09,350.0
"""Financial Services""","""Credit Rating Agencies""",1,0.07,0.05,82.0,27.37,140.0
"""Forest Materials""","""Wood Products""",1,-0.0,-0.0,77.0,25.67,NaN


In [7]:
industry_analysis.filter((pl.col("rs_rating_avg_weight") >= 70)).sort(
    "industry_score", descending=True
).filter(pl.col("symbols_count") > 1)

sector_cmaze,basic_industry_cmaze,symbols_count,1_mo_avg_weight_rtr_pct,3_mo_avg_weight_rtr_pct,rs_rating_avg_weight,industry_score,1_by_3_rtr
str,str,u32,f64,f64,f64,f64,f64
"""Metals & Mining""","""Mining/Minerals""",9,20.8,29.69,92.02,47.5,70.0
"""FMCG""","""Food Products""",5,-0.13,47.08,91.51,46.15,-0.0
"""Financial Services""","""NBFC""",9,5.92,33.17,95.87,44.99,18.0
"""Construction""","""Construction Products Miscalla…",3,9.83,31.42,88.53,43.26,31.0
"""Metals & Mining""","""Metal Fabrication""",5,11.19,22.68,92.2,42.02,49.0
…,…,…,…,…,…,…,…
"""Oil Gas & Consumable Fuels""","""Oil & Gas-Refining & Marketing""",3,0.75,1.16,83.66,28.52,65.0
"""IT""","""Software Services""",15,-0.09,8.37,75.62,27.97,-1.0
"""Chemicals""","""Chemicals-Plastics""",2,0.58,0.99,82.15,27.91,59.0
